In [13]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
 
from keras.models import model_from_json
from keras.models import load_model


In [14]:
dataframes = pandas.read_csv("../data/files/testdefinitivo.csv", delimiter=',',error_bad_lines=False,compression='gzip')
dataframes.head(3)

,floor,id,lat-lon,place_name,place_with_parent_names,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2,...,comedor,sum,patio,palier,rooms1,rooms2,surface_total_in_m21,surface_total_in_m22,surface_covered_in_m21,surface_covered_in_m22
0,0.0,3632,6287.862607,11.987854,-6.241196,-0.883849,0.071429,-0.000000,0.037131,0.035609,...,1.0,0.0,0.0,0.0,0.2,0.333333,0.035443,0.035533,0.036458,0.036458
1,0.0,3633,-6472.820151,28.533506,-1.614436,-0.883849,NaN,2.538785,NaN,NaN,...,1.0,1.0,0.0,0.0,0.0,0.000000,-0.005063,-0.005076,-0.005208,-0.005208
2,0.0,2263404,3689.986230,42.399812,-1.984977,-0.883849,0.035714,-0.000000,0.020313,0.021783,...,0.0,1.0,0.0,0.0,0.2,0.333333,0.022785,0.020305,0.020833,0.020833


In [15]:
dataframes.columns.tolist()
test=dataframes[['surface_covered_in_m2', 'place_name', 'place_with_parent_names', 'surface_total_in_m2', 'hidromasaje', 'laundry', 'parquet', 'pileta', 'cochera', 'jacuzzi', 'hall', 'vista', 'educacion', 'solarium', 'parrilla', 'antigua', 'esquina', 'patio', 'surface_total_in_m21']]

test.fillna(test.mean(),inplace= True)
test.head(10)

,surface_covered_in_m2,place_name,place_with_parent_names,surface_total_in_m2,hidromasaje,laundry,parquet,pileta,cochera,jacuzzi,hall,vista,educacion,solarium,parrilla,antigua,esquina,patio,surface_total_in_m21
0,0.037131,11.987854,-6.241196,0.035609,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035443
1,0.044427,28.533506,-1.614436,0.072466,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.005063
2,0.020313,42.399812,-1.984977,0.021783,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.022785
3,0.015625,-42.517241,-67.410441,0.014691,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.015190
4,0.021875,-42.517241,-67.410441,0.020770,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.020253
5,0.015625,-42.517241,-67.410441,0.014691,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.015190
6,0.015625,-42.517241,-67.410441,0.014691,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.015190
7,0.015625,-42.517241,-67.410441,0.014691,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.015190
8,0.030208,-3.028289,-72.037559,0.028875,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.027848
9,0.019792,42.980368,49.704325,0.018744,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.017722


In [16]:
xtest = test.values
 
xtest=xtest[:,0:19]

In [17]:
xtest 

array([[  3.71308855e-02,   1.19878535e+01,  -6.24119595e+00, ...,
          0.00000000e+00,   0.00000000e+00,   3.54430380e-02],
       [  4.44267460e-02,   2.85335057e+01,  -1.61443632e+00, ...,
          0.00000000e+00,   0.00000000e+00,  -5.06329114e-03],
       [  2.03125000e-02,   4.23998116e+01,  -1.98497737e+00, ...,
          0.00000000e+00,   0.00000000e+00,   2.27848101e-02],
       ..., 
       [  6.56250000e-02,   1.92807428e+01,   3.29115211e+01, ...,
          0.00000000e+00,   0.00000000e+00,   1.06329114e-01],
       [  2.03125000e-02,  -2.92739767e+01,   1.64150838e+01, ...,
          0.00000000e+00,   0.00000000e+00,   2.53164557e-02],
       [  6.30208333e-02,   1.56192406e+01,  -4.61370526e+00, ...,
          1.00000000e+00,   0.00000000e+00,   1.79746835e-01]])

In [18]:
 

#model = model_from_json(open('model.json').read())
#model.load_weights('model_weights.h5')
#model.compile(loss='mean_squared_error', optimizer='adam')



In [19]:
print("Cargando modelo...")

model = load_model('model_train.h5')
 

print("")
print("Cargando modelo... OK")

Cargando modelo...

Cargando modelo... OK


In [20]:
print("")
print("Prediciendo valores...")

predictions = model.predict(xtest )


Prediciendo valores...


In [21]:
ids=dataframes['id'].values
precios = pandas.DataFrame(ids ,columns={'id'})
precios.head(5)

,id
0,3632
1,3633
2,2263404
3,2263405
4,2263406


In [22]:
precios['price_usd']=predictions

In [23]:
precios

,id,price_usd
0,3632,4.830890e+05
1,3633,2.641223e+05
2,2263404,1.204699e+05
3,2263405,1.504588e+05
4,2263406,1.582823e+05
5,2263407,1.504588e+05
6,2263408,1.504588e+05
7,2263409,1.504588e+05
8,2263410,2.253760e+05
9,2263411,1.056877e+05


In [24]:
precios.to_csv("../data/files/resultados6.csv",index=False)